In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd, gc
import matplotlib.pyplot as plt
import time
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import cv2, matplotlib.pyplot as plt
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB2, ResNet50V2
print('RAPIDS',cuml.__version__)
print('TF',tf.__version__)

In [ ]:
# RESTRICT TENSORFLOW TO 1GB OF GPU RAM
# SO THAT WE HAVE 15GB RAM FOR RAPIDS
LIMIT = 1
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*LIMIT)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    #print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    print(e)
print('We will restrict TensorFlow to max %iGB GPU RAM'%LIMIT)
print('then RAPIDS can use %iGB GPU RAM'%(16-LIMIT))

In [ ]:
# train = pd.read_csv('/kaggle/input/shopee-product-matching/train.csv')
# train.head()

In [ ]:
test = pd.read_csv('/kaggle/input/shopee-product-matching/test.csv')
test.head()

# Data Generator

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, df, img_size=256, batch_size=32, path=''): 
        self.df = df
        self.img_size = img_size
        self.batch_size = batch_size
        self.path = path
        self.indexes = np.arange( len(self.df) )
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        ct = len(self.df) // self.batch_size
        ct += int(( (len(self.df)) % self.batch_size)!=0)
        return ct

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X = self.__data_generation(indexes)
        return X
            
    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' 
        X = np.zeros((len(indexes),self.img_size,self.img_size,3),dtype='float32')
        df = self.df.iloc[indexes]
        for i,(index,row) in enumerate(df.iterrows()):
            img = cv2.imread(self.path+row.image)
            X[i,] = cv2.resize(img,(self.img_size,self.img_size)) #/128.0 - 1.0
        return X

# Image Embeddings

In [ ]:
# BASE = '../input/shopee-product-matching/train_images/'

# WGT = '../input/efficientnetb0-notoph5/efficientnetb0_notop.h5'
# model = EfficientNetB0(weights=WGT,include_top=False, pooling='avg', input_shape=None)
# #model = ResNet50V2(weights=WGT,include_top=False, pooling='avg', input_shape=None)

# embeds = []
# CHUNK = 1024*4

# print('Computing image embeddings...')
# CTS = len(train)//CHUNK
# if len(train)%CHUNK!=0: CTS += 1
# for i,j in enumerate( range( CTS ) ):
    
#     a = j*CHUNK
#     b = (j+1)*CHUNK
#     b = min(b,len(train))
#     print('chunk',a,'to',b)
    
#     test_gen = DataGenerator(train.iloc[a:b], batch_size=32, path=BASE)
#     image_embeddings = model.predict(test_gen,verbose=1,use_multiprocessing=True, workers=4)
#     embeds.append(image_embeddings)
    
# del model
# _ = gc.collect()
# image_embeddings = np.concatenate(embeds)
# del embeds
# print('image embeddings shape',image_embeddings.shape)

# Nearest Neighbors

In [ ]:
# model = NearestNeighbors(n_neighbors = 50, metric = 'cosine')
# model.fit(image_embeddings)

# n_distances = []
# n_indices = []
# CHUNK = 1024*4

# print('Computing image nearest neighbors...')
# CTS = len(train)//CHUNK
# if len(train)%CHUNK!=0: CTS += 1
# for i in range( CTS ):
    
#     # calculates chunk
#     a = i*CHUNK
#     b = (i+1)*CHUNK
#     b = min(b,len(train))
#     print('chunk',a,'to',b)
    
#     distances, indices_kNN = model.kneighbors(image_embeddings[a:b,])
#     n_distances.append(distances)
#     n_indices.append(indices_kNN)
    
#     del distances, indices_kNN
    
# _ = gc.collect() # garbage collector
# n_distances = np.concatenate(n_distances) # convert sequence of arrays to ndarray
# n_indices = np.concatenate(n_indices)
# print('distances shape', np.shape(n_distances))
# print('indices shape', np.shape(n_indices))

In [ ]:
# for ind in range(50):
#     plt.hist(n_distances[ind, :], color = "skyblue")
    
# plt.show()

It looks like 0.15 is a good cutoff for neighbor distances.

# Text Normalization

In [ ]:
# lowercase = [title.lower() for title in train.title]
# tokenized = [word_tokenize(title) for title in lowercase]

# stopwords_punctuation = stopwords.words('english')
# punctuation = [ '!', '"', '#', '$', '%', '&', '(', ')', '*', '+', '-', '.', '/',  '\\', ':', ';', '<', '=', '>',
#            '?', '@', '[', ']', '^', '_', '`', '{', '|', '}', '\t','\n',"'",",",'~' , '—']
# stopwords_punctuation.extend(punctuation)

# titles_no_stopwords = []
# for title in tokenized:
#     title_no_sw = [word for word in title if not word in stopwords_punctuation]
#     title_string = ' '.join(title_no_sw)
#     titles_no_stopwords.append(title_string)
    
# train['title_no_sw'] = titles_no_stopwords
# train.head()

# Make Predictions

In [ ]:
# start_time = time.time()

# ids = []
# item_group = [] # tracks which group an item is in
# groups = {} # tracks which items are in each group
# for ind in train.index:
    
#     indices_phash = train.loc[(train['image_phash'] == train['image_phash'][ind])].index 
#     indices_title = train.loc[(train['title_no_sw'] == train['title_no_sw'][ind])].index 
#     indices_image = n_indices[ind, np.where(n_distances[ind] < 0.15)[0]]
#     indices = set(indices_phash).union(set(indices_title)).union(set(indices_image))
    
#     # check if an item this matches already has a group
#     match_ids = list(train.loc[indices]['posting_id'])
#     if set(match_ids).intersection(ids):
#         existing_group = item_group[ids.index(list(set(match_ids).intersection(ids))[0])]
#         item_group.append(existing_group)
#         groups[existing_group] = set(groups[existing_group]).union(match_ids)
#     else:
#         item_group.append(len(groups))
#         groups[len(groups)] = match_ids
        
#     ids.append(train['posting_id'][ind])
    
# matches = [' '.join(list(groups[ind])) for ind in item_group]

# print(time.time() - start_time)

# Training Score

In [ ]:
# # add column with ground truth
# tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
# train['target'] = train.label_group.map(tmp)

# train['oof'] = [list(groups[ind]) for ind in item_group]
# train.head()

In [ ]:
# def getMetric(col):
#     def f1score(row):
#         n = len( np.intersect1d(row.target,row[col]) )
#         return 2*n / (len(row.target)+len(row[col]))
#     return f1score

# train['f1'] = train.apply(getMetric('oof'),axis=1)
# print('CV score =',train.f1.mean())

Using EfficientNetB1 resulted in a train score of 0.6521957366521641, and EfficientNetB2 resulted in 0.6546894934201912. ResNet50V2 gave 0.5910695245925622. Using EfficientNetB0 again with the NearestNeighbors metric set to cosine gave a training score of 0.6788261580490697.

# Applying to test set

In [ ]:
BASE = '../input/shopee-product-matching/test_images/'

WGT = '../input/effnetb0/efficientnetb0_notop.h5'
model = EfficientNetB0(weights=WGT,include_top=False, pooling='avg', input_shape=None)

embeds = []
CHUNK = 1024*4

print('Computing image embeddings...')
CTS = len(test)//CHUNK
if len(test)%CHUNK!=0: CTS += 1
for i,j in enumerate( range( CTS ) ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(test))
    print('chunk',a,'to',b)
    
    test_gen = DataGenerator(test.iloc[a:b], batch_size=32, path=BASE)
    image_embeddings = model.predict(test_gen,verbose=1,use_multiprocessing=True, workers=4)
    embeds.append(image_embeddings)
    
del model
_ = gc.collect()
image_embeddings = np.concatenate(embeds)
del embeds
print('image embeddings shape',image_embeddings.shape)

In [ ]:
neighbors = 3
if test.shape[0] > 3:
    neighbors = 50
model = NearestNeighbors(n_neighbors = neighbors, metric = 'cosine')
model.fit(image_embeddings)

n_distances = []
n_indices = []
CHUNK = 1024*4

print('Computing image nearest neighbors...')
CTS = len(test)//CHUNK
if len(test)%CHUNK!=0: CTS += 1
for i in range( CTS ):
    
    # calculates chunk
    a = i*CHUNK
    b = (i+1)*CHUNK
    b = min(b,len(test))
    print('chunk',a,'to',b)
    
    # find 50 neighbors for everything in this chunk
    distances, indices_kNN = model.kneighbors(image_embeddings[a:b,])
    n_distances.append(distances)
    n_indices.append(indices_kNN)
    
    del distances, indices_kNN
    
_ = gc.collect() # garbage collector
n_distances = np.concatenate(n_distances) # convert sequence of arrays to ndarray
n_indices = np.concatenate(n_indices)
print('distances shape', np.shape(n_distances))
print('indices shape', np.shape(n_indices))

In [ ]:
lowercase = [title.lower() for title in test.title]
tokenized = [word_tokenize(title) for title in lowercase]

stopwords_punctuation = stopwords.words('english')
punctuation = [ '!', '"', '#', '$', '%', '&', '(', ')', '*', '+', '-', '.', '/',  '\\', ':', ';', '<', '=', '>',
           '?', '@', '[', ']', '^', '_', '`', '{', '|', '}', '\t','\n',"'",",",'~' , '—']
stopwords_punctuation.extend(punctuation)

titles_no_stopwords = []
for title in tokenized:
    title_no_sw = [word for word in title if not word in stopwords_punctuation]
    title_string = ' '.join(title_no_sw)
    titles_no_stopwords.append(title_string)
    
test['title_no_sw'] = titles_no_stopwords
test.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as pl
start_time = time.time()
tfidf  = TfidfVectorizer()
corpus = tfidf.fit_transform(test['title_no_sw'])
print(corpus.shape)

In [ ]:
start_time = time.time()

ids = []
item_group = [] # tracks which group an item is in
groups = {} # tracks which items are in each group
for ind in test.index:
    indices_phash = test.loc[(test['image_phash'] == test['image_phash'][ind])].index 
    indices_title = test.loc[(test['title_no_sw'] == test['title_no_sw'][ind])].index 
    indices_image = n_indices[ind, np.where(n_distances[ind] < 0.15)[0]]
    alike_titles = []
    for j in test.index:
        if ind != j and cosine_similarity(corpus[ind], corpus[j]) > .75:
            alike_titles.append(j)
    indices = set(indices_phash).union(set(indices_title)).union(set(indices_image)).union(alike_titles)
    
    # check if an item this matches already has a group
    match_ids = list(test.loc[indices]['posting_id'])
    if set(match_ids).intersection(ids):
        existing_group = item_group[ids.index(list(set(match_ids).intersection(ids))[0])]
        item_group.append(existing_group)
        groups[existing_group] = set(groups[existing_group]).union(match_ids)
    else:
        item_group.append(len(groups))
        groups[len(groups)] = match_ids
        
    ids.append(test['posting_id'][ind])
    
matches = [' '.join(list(groups[ind])) for ind in item_group]

print(time.time() - start_time)

In [ ]:
#now we have the ones that are the same but going to add similar ones as well
print(len(ids))
print(len(matches))

In [ ]:
submission = pd.DataFrame({'posting_id': ids, 'matches': matches})
submission.to_csv('submission.csv', index = False)
submission